### 计算误差 直接计算

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from astropy.time import Time 
namelist = ['2003MA3','2010CO1','2010DG77','2010EX11','2010FC81','2010FH81','2010GS7','2010JN71','2010KX7','2010PW58','2010XP69','2019DD2']
nlist = ['1994cb','1996gq','2002lt24', '2008np3', '2009ux17' ,'2010aj30', '2010bh115' ,'2010ca55' ,'2010cc55' ,'2010cn141','2010dh77' ,'2010fa81' ,'2010gp67' ,'2010hw81' ,'2010hx107', '2010hz104' ,'2010jx16', '2010kp10' ,'2010ku7' ,'2010lh14' ,'2010lj68' ,'2010lk68' ,'2010ll68' ,'2010lv108' ,'2010ns36', '2010ny65', '2010oh126', '2010pu66' ,'2014fd7' ,'2015tf' ,'2016uh101']
alist = namelist + nlist
print(alist)

['2003MA3', '2010CO1', '2010DG77', '2010EX11', '2010FC81', '2010FH81', '2010GS7', '2010JN71', '2010KX7', '2010PW58', '2010XP69', '2019DD2', '1994cb', '1996gq', '2002lt24', '2008np3', '2009ux17', '2010aj30', '2010bh115', '2010ca55', '2010cc55', '2010cn141', '2010dh77', '2010fa81', '2010gp67', '2010hw81', '2010hx107', '2010hz104', '2010jx16', '2010kp10', '2010ku7', '2010lh14', '2010lj68', '2010lk68', '2010ll68', '2010lv108', '2010ns36', '2010ny65', '2010oh126', '2010pu66', '2014fd7', '2015tf', '2016uh101']


In [9]:
name = alist[0].lower()
print(name)
rawdata = pd.read_csv(f'./WisePHA2010/wisecsv/{name}.csv')
rawdata = rawdata.loc[:,['mjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro','w3mpro','w3sigmpro','w4mpro','w4sigmpro','ph_qual','sso_flg']]
rawdata.insert(loc=len(rawdata.columns),column='tdbjd',value='no')
rawdata.insert(loc=len(rawdata.columns),column='tdbmjd',value='no')
#rawdata['mjd'] = 2400000.5+rawdata['mjd']
for i in tqdm(range(len(rawdata))):  
    t = Time(rawdata['mjd'][i],format='mjd',scale='utc')
    t.format = 'iso'
    rawdata['tdbjd'][i] = t.tdb.jd
    rawdata['tdbmjd'][i] = t.tdb.mjd
rawdata
data = rawdata.loc[:,['tdbjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro','w3mpro','w3sigmpro','w4mpro','w4sigmpro']]
data1234 = data.loc[:,['w1mpro','w2mpro','w3mpro','w4mpro']]
data1234

2003ma3


  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_4080231/299576267.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawdata['tdbjd'][i] = t.tdb.jd
/tmp/ipykernel_4080231/299576267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawdata['tdbmjd'][i] = t.tdb.mjd
100%|██████████| 8/8 [00:00<00:00, 37.55it/s]


,w1mpro,w2mpro,w3mpro,w4mpro
0,12.717,12.844,10.859,7.791
1,13.366,14.293,11.076,6.999
2,14.639,15.108,11.271,7.775
3,14.801,14.719,10.571,7.226
4,15.575,14.947,10.076,6.937
5,15.408,15.071,11.162,7.485
6,16.816,14.859,8.848,6.480
7,16.723,14.936,8.869,6.789


In [10]:
datafill = data1234.fillna(method='ffill',axis=1).fillna(method='backfill',axis=1)
datafill

,w1mpro,w2mpro,w3mpro,w4mpro
0,12.717,12.844,10.859,7.791
1,13.366,14.293,11.076,6.999
2,14.639,15.108,11.271,7.775
3,14.801,14.719,10.571,7.226
4,15.575,14.947,10.076,6.937
5,15.408,15.071,11.162,7.485
6,16.816,14.859,8.848,6.480
7,16.723,14.936,8.869,6.789


## M2Flux


In [11]:
#-------initial para-------------------------
#Fv0 of W1 ,W2, W3, W4
Fv0 = [-1,306.682,170.663,29.045,8.284]
#Color corrections  Col[Bv=1:8=(100,141,200,283,400,566,800,1131),fc=1:4=(W1,W2,W3,W4)]
Fc0 = np.zeros((1,8))#W1,W2,W3,W4
Fc1 = np.array([-1,17.2062,3.9096,2.6588,1.0032]) #Bv=100
Fc2 = np.array([-1,4.0882,1.9739,1.4002,0.9852])  #Bv=141
Fc3 = np.array([-1,2.0577,1.3448,1.0006,0.9833])  #Bv=200
Fc4 = np.array([-1,1.3917,1.1124,0.8791,0.9865])  #Bv=283
Fc5 = np.array([-1,1.1316,1.0229,0.8622,0.9903])  #Bv=400
Fc6 = np.array([-1,1.0263,0.9919,0.8833,0.9935])  #Bv=566
Fc7 = np.array([-1,0.9884,0.9853,0.9125,0.9958])  #Bv=800
Fc8 = np.array([-1,0.9801,0.9877,0.9386,0.9975])  #Bv=1131
Fc = [Fc0,Fc1,Fc2,Fc3,Fc4,Fc5,Fc6,Fc7,Fc8]
Fc = np.array(Fc)
Bv = 4
Const = np.array([-1,Fv0[1],Fv0[2],Fv0[3],Fv0[4]])
def h2flux(Fconst,dat_H):
    return Fconst*np.power(10,-dat_H/2.5)*1000

/tmp/ipykernel_4080231/3900039758.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Fc = np.array(Fc)


In [12]:
#data of W1,W2,W3,W4
dat_w1 = np.array(datafill['w1mpro'])
#dat_w1= np.hstack([dat_w1,dat_w1*0.1])
dat_w2 = np.array(datafill['w2mpro'])
#dat_w2= np.hstack([dat_w2,dat_w2*0.1])
dat_w3 = np.array(datafill['w3mpro'])
#dat_w3= np.hstack([dat_w3,dat_w3*0.1])
dat_w4 = np.array(datafill['w4mpro'])
#dat_w4= np.hstack([dat_w4,dat_w4*0.1])
#dat w1
dat_fluxw1 = dat_w1
dat_fluxw1 = Const[1]*np.power(10,-dat_w1/2.5)*1000 
#dat w2
dat_fluxw2 = dat_w2
dat_fluxw2 = Const[2]*np.power(10,-dat_w2/2.5)*1000 
#dat w3
dat_fluxw3 = dat_w3
dat_fluxw3 = Const[3]*np.power(10,-dat_w3/2.5)*1000 
#print(f'{Const[1]}*{np.power(10,-dat_w3[0]/2.5)}*1000={dat_fluxw3[0]}')
#dat w4
dat_fluxw4 = dat_w4
dat_fluxw4 = Const[4]*np.power(10,-dat_w4/2.5)*1000 
dat234 = np.concatenate([dat_fluxw2,dat_fluxw3,dat_fluxw4],axis=0)
#np.savetxt(f'obs.txt.{name}',dat234)
dat1234 = np.concatenate([dat_fluxw1,dat_fluxw2,dat_fluxw3,dat_fluxw4],axis=0)
np.savetxt(f'obsNEW.txt.{name}',dat1234)
dat1234

array([ 2.51124038,  1.38130087,  0.42764981,  0.36837342,  0.18058775,
        0.21061451,  0.0575822 ,  0.06273188,  1.24319535,  0.327295  ,
        0.15450387,  0.22107487,  0.17920057,  0.15985984,  0.19432979,
        0.18102534,  1.31665314,  1.07813062,  0.9008892 ,  1.71660898,
        2.70814168,  0.99602762,  8.39211412,  8.23135592,  6.3363636 ,
       13.14135327,  6.43043111, 10.66203467, 13.91361847,  8.39924192,
       21.19532549, 15.94557154])

In [7]:
for i in range(len(alist)):
    name = alist[i].lower()
    print(name)
    rawdata = pd.read_csv(f'{name}.csv')
    rawdata = rawdata.loc[:,['mjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro','w3mpro','w3sigmpro','w4mpro','w4sigmpro','ph_qual','sso_flg']]
    rawdata.insert(loc=len(rawdata.columns),column='tdbjd',value='no')
    rawdata.insert(loc=len(rawdata.columns),column='tdbmjd',value='no')
    #rawdata['mjd'] = 2400000.5+rawdata['mjd']
    for i in tqdm(range(len(rawdata))):  
        t = Time(rawdata['mjd'][i],format='mjd',scale='utc')
        t.format = 'iso'
        rawdata['tdbjd'][i] = t.tdb.jd
        rawdata['tdbmjd'][i] = t.tdb.mjd
    rawdata
    data = rawdata.loc[:,['tdbjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro','w3mpro','w3sigmpro','w4mpro','w4sigmpro']]
    data1234 = data.loc[:,['w1mpro','w2mpro','w3mpro','w4mpro']]
    datafill = data1234.fillna(method='ffill',axis=1).fillna(method='backfill',axis=1)
    
    #data of W1,W2,W3,W4
    dat_w1 = np.array(datafill['w1mpro'])
    #dat_w1= np.hstack([dat_w1,dat_w1*0.1])
    dat_w2 = np.array(datafill['w2mpro'])
    #dat_w2= np.hstack([dat_w2,dat_w2*0.1])
    dat_w3 = np.array(datafill['w3mpro'])
    #dat_w3= np.hstack([dat_w3,dat_w3*0.1])
    dat_w4 = np.array(datafill['w4mpro'])
    #dat_w4= np.hstack([dat_w4,dat_w4*0.1])
    #dat w1
    dat_fluxw1 = dat_w1
    dat_fluxw1 = Const[1]*np.power(10,-dat_w1/2.5)*1000 
    #dat w2
    dat_fluxw2 = dat_w2
    dat_fluxw2 = Const[2]*np.power(10,-dat_w2/2.5)*1000 
    #dat w3
    dat_fluxw3 = dat_w3
    dat_fluxw3 = Const[3]*np.power(10,-dat_w3/2.5)*1000 
    #print(f'{Const[1]}*{np.power(10,-dat_w3[0]/2.5)}*1000={dat_fluxw3[0]}')
    #dat w4
    dat_fluxw4 = dat_w4
    dat_fluxw4 = Const[4]*np.power(10,-dat_w4/2.5)*1000 
    #dat234 = np.concatenate([dat_fluxw2,dat_fluxw3,dat_fluxw4],axis=0)
    #np.savetxt(f'obs.txt.{name.upper()}',dat234)
    dat1234 = np.concatenate([dat_fluxw1,dat_fluxw2,dat_fluxw3,dat_fluxw4],axis=0)
    np.savetxt(f'obsNEW.txt.{name.upper()}',dat1234)   
    

2003ma3


  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_3611034/1447516686.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawdata['tdbjd'][i] = t.tdb.jd
/tmp/ipykernel_3611034/1447516686.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawdata['tdbmjd'][i] = t.tdb.mjd
100%|██████████| 8/8 [00:00<00:00, 1069.87it/s]


2010co1


100%|██████████| 32/32 [00:00<00:00, 1459.08it/s]


2010dg77


...

100%|██████████| 18/18 [00:00<00:00, 1428.93it/s]


2015tf


100%|██████████| 6/6 [00:00<00:00, 1050.46it/s]


2016uh101


100%|██████████| 13/13 [00:00<00:00, 1175.66it/s]
